In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_summary import DataFrameSummary
import shap
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
%matplotlib inline
sns.set()

In [2]:
df_id = pd.read_csv("data/train_identity.csv")
df_t = pd.read_csv("data/train_transaction.csv")

In [3]:
df_id.shape, df_t.shape

((144233, 41), (590540, 394))

In [9]:
df_t.head().T

,0,1,2,3,4
TransactionID,2987000,2987001,2987002,2987003,2987004
isFraud,0,0,0,0,0
TransactionDT,86400,86401,86469,86499,86506
TransactionAmt,68.5,29,59,50,50
ProductCD,W,W,W,W,H
card1,13926,2755,4663,18132,4497
card2,NaN,404,490,567,514
card3,150,150,150,150,150
card4,discover,mastercard,visa,mastercard,mastercard
card5,142,102,166,117,102


In [7]:
sumdf = DataFrameSummary(df_t).summary()

In [8]:
sumdf.T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
TransactionID,590540,3.28227e+06,170474,2.987e+06,3.13463e+06,3.28227e+06,3.4299e+06,3.57754e+06,590540,590540,0,0%,numeric
isFraud,590540,0.03499,0.183755,0,0,0,0,1,590540,2,0,0%,bool
TransactionDT,590540,7.37231e+06,4.61722e+06,86400,3.02706e+06,7.30653e+06,1.12466e+07,1.58111e+07,590540,573349,0,0%,numeric
TransactionAmt,590540,135.027,239.163,0.251,43.321,68.769,125,31937.4,590540,20902,0,0%,numeric
ProductCD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,590540,5,0,0%,categorical
card1,590540,9898.73,4901.17,1000,6019,9678,14184,18396,590540,13553,0,0%,numeric
card2,581607,362.555,157.793,100,214,361,512,600,581607,500,8933,1.51%,numeric
card3,588975,153.195,11.3364,100,150,150,150,231,588975,114,1565,0.27%,numeric
card4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588963,4,1577,0.27%,categorical
card5,586281,199.279,41.2445,100,166,226,226,237,586281,119,4259,0.72%,numeric


In [11]:
cat_cols_id = ["DeviceType", "DeviceInfo"]
cat_cols_id += ["id"+str(i) for i in range(12,39)]
cat_cols_id

['DeviceType',
 'DeviceInfo',
 'id12',
 'id13',
 'id14',
 'id15',
 'id16',
 'id17',
 'id18',
 'id19',
 'id20',
 'id21',
 'id22',
 'id23',
 'id24',
 'id25',
 'id26',
 'id27',
 'id28',
 'id29',
 'id30',
 'id31',
 'id32',
 'id33',
 'id34',
 'id35',
 'id36',
 'id37',
 'id38']

In [10]:
cat_cols_t = ["ProductCD","emaildomain","addr1", "addr2", "P_emaildomain", "R_emaildomain"]
cat_cols_t += ["card"+str(i) for i in range(1,7)]
cat_cols_t += ["M"+str(i) for i in range(1,10)]
cat_cols_t

['ProductCD',
 'emaildomain',
 'addr1',
 'addr2',
 'P_emaildomain',
 'R_emaildomain',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']